# Autopreise

Die Daten stammen von https://data.world/data-society/used-cars-data
Datensätze aus dem Internet darf man nur unter Berücksichtung geltenden Rechts weiterverbreiten.
Wenn Sie dieses Jupyter Notebook später zu Hause durcharbeiten möchten, müssen Sie sich bei data.world einen Account anlegen und selbst die Daten von dort beziehen.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model

In [ ]:
df = pd.read_csv("autos.csv", encoding="ISO-8859-1")
df.info()
df.head()

In [ ]:
df_kleinwagen = df[df["vehicleType"] == "kleinwagen"]
df_kleinwagen.info()
df_kleinwagen.head()

In [ ]:
df_kleinwagen.describe()

## Verteilung von Kilometerstand

In [ ]:
df_kleinwagen["kilometer"].hist()
plt.xlabel("Kilometer")
plt.ylabel("Anzahl")
plt.show()

Der Kilometerstand sieht glaubwürdig aus

## Verteilung vom Kaufpreis

In [ ]:
ax = df_kleinwagen["price"].hist()
ax.ticklabel_format(style='plain')
plt.xlabel("Preis")
plt.ylabel("Anzahl")
plt.show()

Sehr wenige unverhältnismäßig teure Autos führen dazu, dass man nur einen blauen Balken sehen kann.
Für die übrigen Preisgruppen sind die Balken so klein, dass man sie nicht erkennen kann.

Diese Preisverteilung und insbesondere die sehr hohen Preise für Gebrauchtautos sehen nicht glaubwürdig aus.
Da tauchen wir einmal tiefer ein.
Wie viele gebrauchte Autos sollen über 50.000 € kosten?

In [ ]:
df_kleinwagen[df_kleinwagen["price"] > 50000]

Mit dem Tilde-Operator negieren wir den Suchausdruck.
Wir schließen alle Autos aus, die angeblich über 20.000€ kosten sollen.

In [ ]:
df_kleinwagen = df_kleinwagen[~(df_kleinwagen["price"] > 20000)]
df_kleinwagen.info()
df_kleinwagen.describe()

In [ ]:
df_kleinwagen["price"].hist()
plt.xlabel("Preis")
plt.ylabel("Anzahl")
plt.show()

## Auswahl von Automarke und -modell

Wir wollen ja nicht Äpfel mit Birnen vergleichen, deswegen legen wir uns nun fest.
Damit es möglichst repräsentativ ist, nehmen wir das am häufigsten vorkommende Auto.

In [ ]:
autotypen = df_kleinwagen["name"].value_counts()
autotypen

In [ ]:
df_ford_fiesta = df_kleinwagen[df_kleinwagen["name"] == "Ford_Fiesta"]
df_ford_fiesta.describe()

## Visualisiere Korrelationen

In [ ]:
sns.lmplot(x='kilometer', y='price', data=df_ford_fiesta, aspect=1.2, line_kws={"color": "green"})
plt.ylabel("Preis (€)")
plt.xlabel("Kilometerstand (km)")
plt.show()

In [ ]:
df_ford_fiesta[['kilometer','price']].corr()

In [ ]:
sns.lmplot(x='yearOfRegistration', y='price', data=df_ford_fiesta, aspect=1.2, line_kws={"color": "green"})
plt.ylabel("Preis (€)")
plt.xlabel("Jahr der Erstzulassung")
plt.show()

In [ ]:
df_ford_fiesta[['yearOfRegistration','price']].corr()

## Berechne Lineare Regression

Diese bezieht nun beide Faktoren, sowohl Kilometer als auch das Jahr der Anmeldung, mit ein.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df_ford_fiesta[
    ["kilometer", "yearOfRegistration"]], df_ford_fiesta["price"])

In [ ]:
lm = sklearn.linear_model.LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

Die Differenz zwischen der Vorhersage und dem tatsächlichen Preis untersuchen wir genauer.
Dafür erstellen wir einen eigenen DataFrame.

In [ ]:
difference = pd.DataFrame(data={
    "y_test": y_test.values, 
    "y_pred": y_pred})

difference

In [ ]:
difference.describe()

Die Differenz zwischen Vorhersage und tatsächlichem Preis lässt sich mit einem Histogramm visualisieren.

In [ ]:
(difference["y_test"] - difference["y_pred"]).plot.hist()
plt.xlabel("Preisdifferenz (€)")
plt.show()